# Création de l'IA détecteur de plagiat
- avant d'exécuter ce notebook, il faut éxécuter le fichier unpack.py

In [1]:
import os
from transformers import BertTokenizer, BertModel
import torch
import faiss
import pickle
import numpy as np
from sentence_transformers import util

/Users/nassimnady/Documents/M2/S9/LLM/LLM_projet_plagiat/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/pyt

On commence par importer toutes les librairies nécessaires pour le transformers

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
# Chemins des dossiers
original = "TestData1/Original"
copie = "TestData1/Copy"


Ajouter expliations

On créé ensuite la fonction qui va générer les embeddings via BERT

In [3]:
def embed_bert(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[0, 1:-1, :].mean(dim=0).numpy()
    return embeddings

Expliquer la fonction

On génère ensuite un embedding de tout le corpus de textes origniaux

In [4]:
corpus = []
corpus_embeddings = []
for filename in os.listdir(original):
    if filename.endswith(".txt"):
        with open(os.path.join(original, filename), 'r') as f:
            texte_original = f.read()
            corpus.append(texte_original)
            corpus_embeddings.append(embed_bert(texte_original, tokenizer, model))

corpus_embeddings = np.array(corpus_embeddings)

FileNotFoundError: [Errno 2] No such file or directory: 'output/source/'

In [22]:
def check_plagiarism(text, tokenizer, model, index, corpus_embeddings, output_file="plagiat.txt", threshold=0.8):
    
    # Calcul de l'embedding du texte à vérifier
    text_embedding = embed_bert(text, tokenizer, model)
    D, I = index.search(np.array([text_embedding]), k=5)  # Recherche des k voisins les plus proches

    # Stocker les textes similaires
    similar_texts = []

    # Ouverture du fichier pour écrire les résultats
    with open(output_file, "w", encoding="utf-8") as file:
        for i in range(len(I[0])):
            # Calcul de la similarité cosinus
            similarity = util.cos_sim(text_embedding, corpus_embeddings[I[0][i]])
            if similarity >= threshold:
                detected_text = corpus[I[0][i]]
                similar_texts.append({
                    'text': detected_text,
                    'similarity': similarity.item()
                })
                # Écriture dans le fichier
                file.write(f"Plagiat détecté :\n")
                file.write(f"- Texte source : {detected_text}\n")
                file.write(f"- Texte Similaire : {similar_texts}\n")
                file.write(f"- Similarité : {similarity.item():.2f}\n\n")

    print(f"Résultats de plagiat enregistrés dans {output_file}")
    return similar_texts

On créé maintenant l'index FAISS puis on parcours le dossier contenant les copies pour trouver les plagiats

In [23]:
# Créer l'index FAISS
index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

# Vérifier le plagiat
for filename in os.listdir(copie):
    if filename.endswith(".txt"):
        with open(os.path.join(copie, filename), 'r') as f:
            texte_copie = f.read()
            resultats = check_plagiarism(texte_copie, tokenizer, model, index, corpus_embeddings)



Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
Résultats de plagiat enregistrés dans plagiat.txt
